In [34]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from sklearn.cluster import KMeans
from sklearn import preprocessing
import re
import nltk
import itertools

In [35]:
import pandas as pd
import numpy as np

In [36]:
fa19 = pd.read_csv('fa19_queries.csv')
fa19.head()

,queryID,variantID,queryText
0,313665,4388082,"select e.empName, e.annualSalary, d.deptName, ..."
1,34820,4388082,"SELECT e.empName, e.annualSalary, d.deptName, ..."
2,325288,4388082,"SELECT e.empName, e.annualSalary, d.deptName, ..."
3,36577,4388082,"SELECT empName, annualSalary, deptName, deptLo..."
4,342776,4388082,"SELECT E.empName, E.annualSalary, D.deptName, ..."


In [37]:
def findAndReplace(ql):
    returning = []
    yes = set()
    for contents in ql:
#             pattern = re.compile(r"\<phrase\>.*?\</phrase\>")
#             underscores = pattern.sub(lambda match: match.group(0).replace(' ','_') ,contents)
#             underscores = re.sub('\<phrase\>|\</phrase\>', '_', underscores)
        lol = nltk.word_tokenize(contents)
#             for haha in lol:
#                 if haha[0] is '_':
#                     yes.add(haha)
        returning.append(lol)
    return returning

In [38]:
queryText = fa19['queryText']
queryText = queryText.str.lower()
queryList = queryText.tolist()
order = fa19['queryID'].tolist()

print(queryList[1])
tokenized = findAndReplace(queryList)
print(tokenized[1])
model = Word2Vec(tokenized, size=100, window=5, workers=12, negative=0, hs=1, sample=1e-3)

select e.empname, e.annualsalary, d.deptname, d.deptlocation, e.jobtitle
from employee e join department d on e.deptid = d.deptid 
where ((d.deptlocation = 'dubai') or (d.deptname = 'sales')) and (e.annualsalary >= 84000 and e.annualsalary <= 120000) and ((e.empname like '_a%' )or (e.empname like '_i%'))
order by e.empname,  d.deptname desc


['select', 'e.empname', ',', 'e.annualsalary', ',', 'd.deptname', ',', 'd.deptlocation', ',', 'e.jobtitle', 'from', 'employee', 'e', 'join', 'department', 'd', 'on', 'e.deptid', '=', 'd.deptid', 'where', '(', '(', 'd.deptlocation', '=', "'dubai", "'", ')', 'or', '(', 'd.deptname', '=', "'sales", "'", ')', ')', 'and', '(', 'e.annualsalary', '>', '=', '84000', 'and', 'e.annualsalary', '<', '=', '120000', ')', 'and', '(', '(', 'e.empname', 'like', "'_a", '%', "'", ')', 'or', '(', 'e.empname', 'like', "'_i", '%', "'", ')', ')', 'order', 'by', 'e.empname', ',', 'd.deptname', 'desc']


In [39]:
model.save("word2vecQueries.model")
model.wv.save_word2vec_format("QueriesNon-embedded.txt", binary=False)

In [40]:
len(model.wv.vocab)

490

In [41]:
print(model.wv.index2word[90])
phrasal = open("Queries Non-embedded.txt", "r")
vocab = []
embeddings = []
both = []
new = []
for idx, word in enumerate(model.wv.index2word):
        sentence = ""
        vocab.append(word)
        sentence += word+" "
        embeddings.append(model.wv.vectors[idx])
        for vector in model.wv.vectors[idx]:
            sentence += str(vector) + " "
        tup = (word, model.wv.vectors[idx])
        both.append(tup)
        new.append(sentence)
m = open("FA19QueryEmbeddings.txt", "w")
m.write(str(embeddings))
n = open("FA19QueryPhraseEmbeddings.txt", "w")
n.write(str(both))
n = open("FormattedFA19Queries.txt", "w")
n.write(str(new))

jobtitle


582284

In [48]:
model.wv.__getitem__('jobtitle')

array([ 0.47491342, -0.15493391,  0.07518594,  0.12307548, -0.408684  ,
       -0.823419  , -0.64922315, -0.6355293 ,  1.2383932 , -0.28111437,
        0.3352338 ,  0.14362024,  0.41936058,  0.27107602, -0.50241774,
        0.3523441 , -0.6891677 ,  0.01738207, -0.9856541 , -1.4893625 ,
       -0.21833646,  0.8109977 , -0.37055102,  0.7093048 ,  0.1481504 ,
       -0.09731897,  0.05349965, -0.7689386 ,  0.4610307 ,  1.3222123 ,
        0.46172556, -0.9502278 , -0.30209482,  0.36303264,  0.60284644,
        0.27670032,  0.06869691, -0.67791075, -0.41695908,  0.7471047 ,
        0.21218635, -0.6393327 , -0.13632007,  0.2925045 , -0.11298273,
        0.19882715,  0.21976532, -0.06650586,  0.91304904, -1.0257984 ,
        0.8099864 ,  0.8455717 ,  0.08734801,  0.11483569, -0.2007701 ,
       -0.8890593 , -0.05127467,  0.6514148 , -0.1990501 ,  0.05705967,
       -0.4110223 , -1.0467361 ,  0.0035191 , -0.03459618,  0.42211223,
       -0.06983773, -0.7091499 , -0.04970587,  0.41123545,  0.07

In [54]:
def get_document_embeddings(df, model):
    corpus = df['queryText'].str.lower().tolist()
    ids=df['queryID'].tolist()
    document_embeddings = []
    doc_emb = dict()
    for i,document in enumerate(corpus):
        word_counts = {}
#         document = document.decode('utf-8', 'ignore')
        document = nltk.word_tokenize(document)
        for word in document:
            if word not in word_counts:
                word_counts[word] = 0
            word_counts[word] += 1
        this_document_embeddings = np.zeros(100)
        total_words = 0 
        for word in word_counts:
#             print(word)
            try:
                this_document_embeddings += word_counts[word] * model.wv.__getitem__(word)
                total_words += word_counts[word]
            except KeyError:
                print (word, 'not in vocabulary')
        this_document_embeddings /= total_words
        document_embeddings.append(this_document_embeddings)
        doc_emb[ids[i]]=this_document_embeddings
    return document_embeddings, doc_emb

In [56]:
doc_emb_list, doc_emb_dict = get_document_embeddings(fa19, model)

100000 not in vocabulary
/12 not in vocabulary
=10000 not in vocabulary
100000 not in vocabulary
70000 not in vocabulary
dept.deptlocation not in vocabulary
emp.jobtitle not in vocabulary
substr not in vocabulary
d.deptname= not in vocabulary
=10000 not in vocabulary
annualsalary/12 not in vocabulary
dep.deptname not in vocabulary
dep.deptlocation not in vocabulary
emp.jobtitle not in vocabulary
dep not in vocabulary
emp.annualsalary/12 not in vocabulary
=7000 not in vocabulary
e1.annualsalary not in vocabulary
d1.deptname not in vocabulary
d1.deptlocation not in vocabulary
e1.jobtitle not in vocabulary
d1.deptid not in vocabulary
e1.deptid not in vocabulary
e1.annualsalary/12 not in vocabulary
94000 not in vocabulary
10000.0 not in vocabulary
7000.0 not in vocabulary
annualsalary/12 not in vocabulary
=10000 not in vocabulary
d.deptlocation='dubai not in vocabulary
d.deptname='sales not in vocabulary
100000 not in vocabulary
7000*12 not in vocabulary
10000*12 not in vocabulary
employee

bahaha not in vocabulary
know not in vocabulary
after not in vocabulary
each not in vocabulary
mathes not in vocabulary
employees not in vocabulary
new.c not in vocabulary
d2.deptid not in vocabulary
d2 not in vocabulary
employee_count not in vocabulary
table not in vocabulary
we not in vocabulary
every not in vocabulary
plus not in vocabulary
their not in vocabulary
info not in vocabulary
aggregate not in vocabulary
function not in vocabulary
i.e not in vocabulary
but not in vocabulary
combined not in vocabulary
with not in vocabulary
d.attribute not in vocabulary
ask not in vocabulary
something not in vocabulary
you not in vocabulary
can not in vocabulary
anything not in vocabulary
clause not in vocabulary
is not in vocabulary
use not in vocabulary
employees not in vocabulary
find not in vocabulary
this not in vocabulary
will not in vocabulary
help not in vocabulary
see not in vocabulary
per not in vocabulary
department='ece not in vocabulary
department='cs' not in vocabulary
ss not 

brandname='samsung not in vocabulary
firstname= not in vocabulary
phonenumber= not in vocabulary
p.productid=productid not in vocabulary
brandname='apple'or not in vocabulary
brandname='google not in vocabulary
subq.productname not in vocabulary
custs.firstname not in vocabulary
purchases.productid=products.productid not in vocabulary
customers.firstname= not in vocabulary
products.brandname= not in vocabulary
custs not in vocabulary
purchases.customerid=custs.customerid not in vocabulary
subq not in vocabulary
samsung.productname not in vocabulary
cu.phonenumber not in vocabulary
other not in vocabulary
samsung.customerid not in vocabulary
other.customerid not in vocabulary
pt.productname not in vocabulary
pt not in vocabulary
pt.brandname not in vocabulary
pt2 not in vocabulary
pt2.brandname not in vocabulary
pur.productid=pro.productid not in vocabulary
pro.brandname='apple not in vocabulary
pro.brandname='google not in vocabulary
p2.brandname='google not in vocabulary
p2.brandname=

newt not in vocabulary
'harry not in vocabulary
'new not in vocabulary
e1.empid not in vocabulary
e2.empname not in vocabulary
'harry not in vocabulary
e1.hiredate not in vocabulary
e3 not in vocabulary
e3.empname not in vocabulary
'new not in vocabulary
s1.grade not in vocabulary
s1.maxsalary not in vocabulary
s1.grade not in vocabulary
s1.maxsalary not in vocabulary
sg1.grade not in vocabulary
sg1 not in vocabulary
sg1.minsalary not in vocabulary
sg1.maxsalary not in vocabulary
newt not in vocabulary
'harry not in vocabulary
'london' not in vocabulary
'new not in vocabulary
york' not in vocabulary
'new not in vocabulary
s1.grade not in vocabulary
s1.maxsalary not in vocabulary
managerid=e.empid not in vocabulary
e1.managerid not in vocabulary
manager.salary not in vocabulary
manager.empid not in vocabulary
t1.empid not in vocabulary
avgsalary not in vocabulary
t2.managerid not in vocabulary
x.salary not in vocabulary
x.managerid not in vocabulary
e1.empid not in vocabulary
e3.salary 

g.name not in vocabulary
g.p2id not in vocabulary
p1id not in vocabulary
p2id not in vocabulary
p1.brandname=p2.brandname not in vocabulary
prodname not in vocabulary
p.brandname=p2.brandname not in vocabulary
result not in vocabulary
counts,0 not in vocabulary
q.productname not in vocabulary
temp.num not in vocabulary
p2.brandname=p.brandname not in vocabulary
name1 not in vocabulary
name2 not in vocabulary
leftjoin+on not in vocabulary
b2 not in vocabulary
b2.brandname not in vocabulary
b1.brandname not in vocabulary
b1 not in vocabulary
x.productname not in vocabulary
x.brandname not in vocabulary
x.yearreleased not in vocabulary
inside not in vocabulary
inside.yearreleased not in vocabulary
inside.brandname not in vocabulary
thecount not in vocabulary
p1.brandname=p2.brandname not in vocabulary
numbers.freq not in vocabulary
final.productname not in vocabulary
freq not in vocabulary
bn1 not in vocabulary
pn1 not in vocabulary
yr1 not in vocabulary
x.productname not in vocabulary
x.

In [61]:
print(doc_emb_dict[313665])

[-1.36768508e-01  2.88506817e-02 -1.02619739e-01  1.55349888e-01
  1.61419740e-01  2.96845551e-02  4.10323939e-02  1.06392086e-01
 -3.35407399e-01  1.47291096e-01 -1.91908884e-01  3.12982467e-02
 -1.13835950e-01  1.88906333e-01  2.70543136e-01 -1.26348692e-02
 -2.11562356e-01 -1.72087770e-01 -1.96837861e-02 -1.54580952e-01
 -2.84423669e-01 -8.98386325e-02  1.10417163e-01 -1.50003803e-01
 -3.33444741e-01  1.81478903e-01  9.95562105e-02 -1.42286765e-01
  1.52650540e-01 -2.50344724e-01  2.35569316e-02 -1.76651022e-01
 -1.76574105e-02  2.78193954e-02  2.77853824e-01 -3.07521110e-01
  1.15021928e-01 -2.23753928e-02  3.85547333e-03  1.56987811e-01
 -2.04564941e-01 -3.29574523e-01 -6.53367126e-02 -2.19808790e-01
 -2.13211161e-01  1.06112329e-01  4.86222635e-02  3.48086989e-01
 -9.05701920e-02 -2.18479345e-01  9.59808159e-03  2.12083967e-01
  2.08196621e-01 -9.33548668e-03  2.77367659e-01 -2.33497120e-01
 -2.69970049e-01 -1.16307950e-01  1.33673980e-01 -2.37969469e-01
 -3.65550243e-01 -2.51383

In [62]:
def write_embeddings(filename, embeddings):
    with open(filename, 'a') as f:
        for document in embeddings:
            for embedding in document:
                f.write(str(embedding) + ' ')
            f.write('\n')

In [63]:
write_embeddings('FA19QueryEmbeddings', doc_emb_list)

In [64]:
import pickle
with open('FA19QueryEmbeddings.pickle', 'wb') as handle:
    pickle.dump(doc_emb_dict, handle)